In [ ]:
from lsst.ts import salobj
from lsst.ts.idl.enums import MTMount
import logging
import asyncio
import os
import yaml

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd
import astropy.units as u

from lsst_efd_client import EfdClient

# Before the tests:

In [ ]:
script = salobj.Controller("Script", index=1)

In [ ]:
mtm_sim = salobj.Remote(script.domain, "MTMount")
await mtm_sim.start_task

In [ ]:
# this is to ensure that MTMount is connected and telemetry is send
mtm_sim_connected = await mtm_sim.evt_connected.aget(timeout=10.) # This is how you get an event/telemetry with await remote.type_name.aget() type being [evt,tel] and name being the name of the topic
#mtm_commandable = connected.command
mtm_sim_telemetry_working = mtm_sim_connected.telemetry

if not mtm_sim_connected or not mtm_sim_telemetry_working:
    raise Exception("Mount not connected or telemetry not being received.")

In [ ]:
#This is to check what the controller state is:
controller_state = await mtm_sim.evt_controllerState.aget()
state = controller_state.controllerState
offline_substate = controller_state.offlineSubstate
print(state)

In [ ]:
#This is to check MTMount is commandable through SAL:
commandable_by_dds = await mtm_sim.evt_commandableByDDS.aget()
dds_state = commandable_by_dds.state
if not dds_state:
    raise Exception("Controller must in CommandableByDDS state.")

In [ ]:
#Print the available MTMount events
print(mtm_sim.salinfo.event_names)

In [ ]:
#Check if MTMount telemetry is getting into the EFD
#Add telemetry as needed
#For the "setCompensateMode" of the Hexapods the "elevation" of the telescope is needed
mtm_sim_Elevation= await mtm_sim.tel_elevation.aget(timeout=10.)
print("Simulated mount elevation", mtm_sim_Elevation.angleActual)

In [ ]:
#Send the MTMount simulator component into STANDBY
await salobj.set_summary_state(mtm_sim, salobj.State.STANDBY)

In [ ]:
#Check if the input to the EFD has stopped
mtm_sim_Elevation= await mtm_sim.tel_elevation.aget(timeout=10.)
print("Simulated mount elevation", mtm_sim_Elevation.angleActual)

In [ ]:
#Stop the script controller
await script.close()

In [ ]:
#Start the MTMount controller
mtm_control = salobj.Controller(name="MTMount")

In [ ]:
#Check if the input to the EFD has started
mtm_control_Elevation= await mtm_control.tel_elevation.aget(timeout=10.)
print("Mount controller elevation", mtm_control_Elevation.angleActual)

# Set the MTMount Controller to the needed values and exectute the tests from another notebook:

In [ ]:
# Set the elevation angle. Do this for every desired elevation angle.
mtm_control.evt_target.set_put(elevation=45)

# After the tests:

In [ ]:
#Stop the MTMount controller
await mtm_control.close()

In [ ]:
#Check that no more telemetry from the MTMount controller is going into the EFD
mtm_control_Elevation= await mtm_contorl.tel_elevation.aget(timeout=10.)
print("Mount controller elevation", mtm_control_Elevation.angleActual)

In [ ]:
#Start the script controller
script = salobj.Controller("Script", index=1)

In [ ]:
#Start the mount Simluator controller
await mtm_sim.start_task

In [ ]:
#Send the MTMount simulator component into ENABLE again
await salobj.set_summary_state(mtm, salobj.State.ENABLE)

In [ ]:
#Check that the EFD is getting MTMount siumlator telemetry

In [ ]:
#print available commands
print(mtm.salinfo.command_names)